In [ ]:
import sys
import os
module_dir = os.path.abspath("/lustrehome/federicacuna/TB_Sept_2023_ml/Code/Pytorch_gnn/")
sys.path.append(module_dir)

In [1]:
import pandas as pd
import torch
import torch_geometric
from torch_geometric.data import Dataset, Data
import numpy as np 
from torch_geometric.loader import DataLoader
import os
from torch_geometric.nn import Sequential, GCNConv,GATConv,SAGEConv
import matplotlib.pyplot as plt
from torch.nn import Linear
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score, accuracy_score
import time
import MyData as data
from matplotlib.lines import Line2D
from torch.nn import Embedding, Linear, ModuleList, ReLU, Sequential


from torch_geometric.loader import DataLoader
from torch_geometric.nn import BatchNorm, PNAConv, global_add_pool
from torch_geometric.utils import degree

from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
view='xz'
lr=1e-4
folder_pkl='optimized'
n_epochs=600

In [ ]:
if "." in str(lr):
    lr_name = str(lr).replace(".", "_")
    print(lr_name)
else:
    lr_name = str(lr)
    print(lr_name)

In [2]:
%%time
data=data.dataset_preparation(root='/lustrehome/federicacuna/TB_Sept_2023_ml/Data/preprocessed/')
if view=='xz':
    fname='pi-_10GeV_4ly_1e-4_inclined_4ly_GAT_viewx'
else:
    fname='pi-_10GeV_4ly_1e-4_inclined_4ly_GAT_viewy'
data_trk=data.get_more_file(0,300,fname)
train_dataset = data_trk[:2000000]
val_dataset =data.get_more_file(300,400,fname)[:400000]
test_dataset =data.get(500,fname)
print('len_train_dataset ',len(train_dataset))
print('len_val_dataset ',len(val_dataset))
print('len_test_dataset ',len(test_dataset))

CPU times: user 11 µs, sys: 0 ns, total: 11 µs
Wall time: 20 µs


In [ ]:
%%time
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device=torch.device('cpu')
print('Using device:', device)
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


In [ ]:
outname=f'NEWDATAbest_model_gatedgenode_lytrk_4_view_{view}_lr_{lr_name}_nepochs_{n_epochs}_len_{len(train_dataset)}.pkl'

In [5]:
%%time
train_loader = DataLoader(train_dataset, batch_size=4032,shuffle=True,pin_memory=True,num_workers=15,drop_last=True) 
val_loader=DataLoader(val_dataset, batch_size=128,shuffle=True,pin_memory=True,num_workers=15,drop_last=True) 
test_loader=DataLoader(test_dataset, batch_size=64,shuffle=True,drop_last=True)

CPU times: user 383 µs, sys: 0 ns, total: 383 µs
Wall time: 392 µs


In [18]:
# Basically the same as the baseline except we pass edge features 
class GDPModel(torch.nn.Module):
    def __init__(self, num_features=5, hidden_size=128,num_class=1,heads=8):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_features = num_features
        self.num_class=num_class
        self.heads=heads
        self.convs1=GATConv(self.num_features, self.hidden_size, edge_dim = 1,heads=self.heads)
        self.convs2=GATConv(self.hidden_size*self.heads, self.num_class, edge_dim = 1,heads=1)
        
        # self.convs = [GATConv(self.num_features, self.hidden_size, edge_dim = 2),
        #               GATConv(self.hidden_size, self.hidden_size, edge_dim = 2)]
        # self.linear = torch.nn.Linear(self.hidden_size, self.target_size)

    def forward(self, data):
        
        x, edge_index, edge_attr, batch= data.x, data.edge_index, data.edge_attr, data.batch
        # print(edge_index.device)
        # print(x.device)
        # print(edge_attr.device)
        # print(batch.device)
        x = self.convs1(x, edge_index,edge_attr)
        x=F.tanh(x)
        x = self.convs2(x, edge_index,edge_attr)
        # for conv in self.convs[:-1]:
        #     x = conv(x, edge_index, edge_attr) # adding edge features here!
        #     x = F.relu(x)
        #     x = F.dropout(x, training=self.training)
        # x = self.convs[-1](x, edge_index, edge_attr) # edge features here as well
        # x = self.linear(x)
        
        
        return F.sigmoid(x) 


In [19]:
%%time
device='cuda'
model = GDPModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
####criterion = torch.nn.CrossEntropyLoss()
criterion=torch.nn.functional.binary_cross_entropy#for sageconv

def train_model(train_data):
    model.to(device)
    # print(model)
    model.train()
    for data in train_data:
        data.to(device)            
        out = model(data) # Perform a single forward pass.        
        labels=data.y.float().to(device)
        loss = criterion(out.squeeze(1).float(), labels)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
    return model.to(device)


CPU times: user 2.81 ms, sys: 1.8 ms, total: 4.61 ms
Wall time: 3.26 ms


In [20]:
threshold=0.8
def evaluate_model(model,test_data):
    model.eval()
    correct = 0
    total_samples = 0
    model_outputs = []
    targets = []
    pred_class=[]

    for data in test_data:
        data.to(device)
        model_output = model(data)
        model_output.to(device)
        predicted_class = torch.where(model_output > threshold, 1, 0)
        
        correct += int((predicted_class == data.y.float()).sum())
        total_samples += len(data.y)

          # store these to get the loss
        model_outputs.extend(model_output.tolist())
        targets.extend(data.y.float().tolist())
        pred_class.extend(predicted_class.tolist())

    
    loss = criterion(torch.tensor(model_outputs).squeeze(1), torch.tensor(targets).float())
    accuracy = accuracy_score(targets, pred_class)
    precision = precision_score(targets, pred_class)
    recall = recall_score(targets, pred_class)
    
    return accuracy, loss, recall, precision 

In [21]:
%%time
train_accuracies = []
validation_accuracies = []
validation_losses = []
train_losses = []

val_recall=[]
val_precision=[]
train_recall=[]
train_precision=[]
times = []
n_ly=4
for epoch in range(1, n_epochs):
    start = time.time()
    print(epoch)
    model = train_model(train_loader)
    print(next(model.parameters()).is_cuda)
    
    train_acc, train_loss, train_rec, train_prec = evaluate_model(model,train_loader)
    train_losses.append(train_loss)
                   
    val_acc, val_loss, val_rec, val_prec = evaluate_model(model,val_loader)
    validation_losses.append(val_loss)

    train_accuracies.append(train_acc)
    validation_accuracies.append(val_acc)
    
    train_precision.append(train_prec)
    train_recall.append(train_rec)
    
    val_precision.append(val_prec)
    val_recall.append(val_rec)

    # save the model if it is the better than any previous ones
    if val_loss.item() <= min(validation_losses).item():
        torch.save(model, f'/lustrehome/federicacuna/TB_Sept_2023_ml/output_gnn_pkl/{folder_pkl}/{outname}')
        print(f'Epoch_stop: {epoch}')
          
    #torch.save(model, "best_model.pkl")
    if epoch % 1 == 0:
        print(f'Epoch: {epoch}, Train Acc: {train_acc:.4f}, Train Loss: {train_loss:.4f}, Val Acc: {val_acc:.4f}, Val Loss: {val_loss:.4f}, time :{time.time() - start}')
    times.append(time.time() - start)
  

1
True
Epoch_stop: 1
Epoch: 1, Train Acc: 0.6917, Train Loss: 0.4160, Val Acc: 0.6946, Val Loss: 0.4114, time :10.48795747756958
2
True
Epoch_stop: 2
Epoch: 2, Train Acc: 0.7812, Train Loss: 0.3590, Val Acc: 0.7828, Val Loss: 0.3551, time :11.065448522567749
3
True
Epoch_stop: 3
Epoch: 3, Train Acc: 0.7762, Train Loss: 0.3382, Val Acc: 0.7764, Val Loss: 0.3346, time :10.11599326133728
4
True
Epoch_stop: 4
Epoch: 4, Train Acc: 0.8212, Train Loss: 0.2894, Val Acc: 0.8243, Val Loss: 0.2840, time :10.953760147094727
CPU times: user 2min 20s, sys: 1.91 s, total: 2min 22s
Wall time: 42.6 s


In [ ]:
best_validation_loss = min(validation_losses)
best_epoch = validation_losses.index(best_validation_loss)
accuracy_at_best_epoch = validation_accuracies[best_epoch]
recall_at_best_epoch=val_recall[best_epoch]
precision_at_best_epoch=val_precision[best_epoch]
print(f"The best result was achieved after {best_epoch} epochs with a validation accuracy of {accuracy_at_best_epoch:.4f} and a loss of {best_validation_loss:.4f}")
print(f"The validation recall is {recall_at_best_epoch:.4f} and the precision is {precision_at_best_epoch:.4f}")

In [ ]:
fig, ax = plt.subplots()
#fig.set_size_inches(30.5, 15.5)

plt.plot(validation_losses,color='orange',label='validation loss')
plt.plot(train_losses,color='blue',label='training loss')

plt.axvline(x=best_epoch, color="green", linewidth=2, linestyle='dashed',label='best epoch')
#plt.legend(['Val Loss', 'Train Loss', "Best Epoch"])
plt.legend()


# Hide the right and top spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.title("Model training and validation loss per epoch", fontsize=10, fontweight='bold')
#plt.ylim(0.05,0.2)
plt.savefig(f'/lustrehome/federicacuna/TB_Sept_2023_ml/output_gnn_img/{folder_pkl}/model_loss_GATedgenode_view_{view}lr_{lr_name}.png')
plt.show()

In [ ]:
fig, ax = plt.subplots()
#fig.set_size_inches(30.5, 15.5)

plt.plot(validation_accuracies,color='orange',label='validation acc')
plt.plot(train_accuracies,color='blue',label='training acc')

plt.legend()

# Hide the right and top spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.title("Model training and validation accuracy per epoch", fontsize=10, fontweight='bold')
# plt.ylim(0.9,1)
plt.savefig(f'/lustrehome/federicacuna/TB_Sept_2023_ml/output_gnn_img/{folder_pkl}/model_acc_GATedgenode_view_{view}lr_{lr_name}.png')
plt.show()

In [ ]:
best_model = torch.load( f'/lustrehome/federicacuna/TB_Sept_2023_ml/output_gnn_pkl/{folder_pkl}/{outname}')
best_model.to(device) 

correct = 0
total_samples = 0
pred_test_cl=[]
targets=[]
with torch.no_grad():
    for data in test_loader:
        data = data.to(device)  
        model_output = best_model(data)
        predicted_class =  torch.where(model_output > 0.8, 1, 0)
        pred_test_cl.extend(predicted_class.tolist())
        correct += int((predicted_class == data.y).sum())
        targets.extend(data.y.tolist())
        total_samples += data.y.size(0)

print('accuracy ',accuracy_score(targets,pred_test_cl),' recall ', recall_score(targets,pred_test_cl),' precision ',precision_score(targets,pred_test_cl))
